In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from transformers import BertTokenizer, AutoModelForSequenceClassification

In [ ]:
model_path = "bert_sentiment_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

label_map = {1:'Positive', 0:'Negative', 2:'Neutral'}

def predict_sentiment(text):
    inputs = tokenizer.encode_plus(
        text,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    predicted_class = torch.argmax(outputs.logits, dim=1)
    sentiment = label_map[predicted_class.item()]
    
    return predicted_class

In [ ]:
test_text = "I dont know about the game rainbow six siege"
predicted_label = predict_sentiment(test_text)
print(f"Sentiment: {predicted_label}")

In [ ]:
true_labels = [1, 0, 2]
predicted_labels = [predict_sentiment(text) for text in test_text]
conf_matrix = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(6, 4))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()